In [ ]:
from ratdata import data_manager as dm, process, ingest
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
beta_low = 11
beta_high = 30

rat_peaks = dict()

for r in dm.RecordingFile.select().where(dm.RecordingFile.condition == 'baseline'):
    filename = Path(r.dirname) / r.filename
    rec = ingest.read_mce_matlab_file(filename)
    if dm.is_recording_rejected(filename.name):
        continue

    fs = int(1/rec.dt)
    
    x = np.mean(rec.electrode_data, 0)
    if dm.is_recording_sliced:
        slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
        start = int(slice.start * fs)
        end = int((slice.start + slice.length) * fs)
        x = x[start:end]

    f, psd = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(f, psd, 2, 100)
    f[0] = 1e-20
    oof = f ** m * (np.e ** b)
    detrended = psd - oof
    idx = np.logical_and(f >= 10, f <= 35)
    peaks = process.find_peaks(f, detrended, beta_low, beta_high)
    if rec.rat_label in rat_peaks:
        rat_peaks[rec.rat_label].append(peaks[0])
    else:
        rat_peaks[rec.rat_label] = [peaks[0]]
    plt.figure(figsize=(12, 6))
    plt.plot(f[idx], detrended[idx])
    plt.title(filename.stem)
    if len(peaks) == 0:
        print('No peaks: %s' % rec.filename)
    else:
        px, py, _ = peaks[0]
        plt.plot(px, py, 'x')
    plt.savefig('plots/peaks/%s_%s.png' % (rec.rat_label, filename.stem), facecolor='white', bbox_inches='tight')
    plt.close()

In [ ]:
%matplotlib inline
c = plt.get_cmap('Dark2')
control_rats = [r.label for r in dm.Rat.select().where(dm.Rat.group == 'control')]
ohda_rats = [r.label for r in dm.Rat.select().where(dm.Rat.group == '6OHDA')]
columns = 2
rows = max([len(control_rats), len(ohda_rats)])

fig, ax = plt.subplots(nrows=rows, ncols=columns, figsize=(12, 16))
for i, rat in enumerate(ohda_rats):
    ax[i, 0].set_title(rat)
    ax[i, 0].set_xlim([11, 25])
    ax[i, 0].set_ylim([-3e-5, 1.5e-4])
    xx, yy, pp = zip(*rat_peaks[rat])
    ax[i, 0].scatter(xx, yy, s=10, color=c.colors[i])
    ax[i, 0].axvline(np.mean(xx), color=c.colors[i], alpha=1)
    ax[i, 0].fill_betweenx(ax[i, 0].get_ylim(), np.ones(2) * (np.mean(xx) - np.std(xx)), np.ones(2) * (np.mean(xx) + np.std(xx)),
                           color=c.colors[i], alpha=0.2)
    # ax[i, 0].axhline(np.mean(yy), color=c.colors[i], alpha=1)
    # ax[i, 0].fill_between(ax[i, 0].get_xlim(), np.ones(2) * (np.mean(yy) - np.std(yy)), np.ones(2) * (np.mean(yy) + np.std(yy)),
    #                       color=c.colors[i], alpha=0.2)

for i, rat in enumerate(control_rats):
    ax[i, 1].set_title(rat)
    ax[i, 1].set_xlim([11, 25])
    ax[i, 1].set_ylim([-3e-5, 1.5e-4])
    xx, yy, pp = zip(*rat_peaks[rat])
    ax[i, 1].scatter(xx, yy, s=10, color=c.colors[len(ohda_rats) + i])
    ax[i, 1].axvline(np.mean(xx), color=c.colors[len(ohda_rats) + i], alpha=1)
    ax[i, 1].fill_betweenx(ax[i, 1].get_ylim(), np.ones(2) * (np.mean(xx) - np.std(xx)), np.ones(2) * (np.mean(xx) + np.std(xx)),
                           color=c.colors[len(ohda_rats) + i], alpha=0.2)
    # ax[i, 1].axhline(np.mean(yy), color=c.colors[len(ohda_rats) + i], alpha=1)
    # ax[i, 1].fill_between(ax[i, 1].get_xlim(), np.ones(2) * (np.mean(yy) - np.std(yy)), np.ones(2) * (np.mean(yy) + np.std(yy)),
    #                       color=c.colors[len(ohda_rats) + i], alpha=0.2)

ax[0, 0].annotate('6-OHDA rats', xy=(0, 0), va='center', ha='center', xycoords=ax[0, 0].title, xytext=(0.5, 2), fontsize=14)
ax[0, 1].annotate('control rats', xy=(0, 0), va='center', ha='center', xycoords=ax[0, 1].title, xytext=(0.5, 2), fontsize=14)
ax[3, 1].axis('off')
fig.suptitle('Most prominent peak location and height', fontsize=14)
fig.tight_layout()
    
